# Análisis Exploratorio y Limpieza de Datos

## 0. Configuración de entorno

Se importan las librerías necesarias para el análisis y la manipulación de datos, y se monta Google Drive para acceder a los archivos.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from google.colab import drive

# Set plotting style
sns.set_theme(style="whitegrid")

# Mount Google Drive
drive.mount('/content/drive')

## 1. Carga de datos y limpieza de nombres de columnas
Se define la ruta del archivo, se carga en un DataFrame y se normalizan inmediatamente los nombres de las columnas.

In [ ]:
file_path = "/content/drive/My Drive/Colab Notebooks/test/rentabilidad_productos.csv"

# Load the dataset
df_raw = pd.read_csv(file_path)

print(f"DataFrame cargado con {df_raw.shape[0]} filas y {df_raw.shape[1]} columnas.")

# --- Initial column name cleaning ---
original_cols = df_raw.columns.to_list()
df_raw.columns = df_raw.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'[^\w\s]', '', regex=True)
cleaned_cols = df_raw.columns.to_list()

print("\n--- Normalización de Nombres de Columnas ---")
col_mapping = pd.DataFrame({'original': original_cols, 'limpio': cleaned_cols})
display(col_mapping)

## 3. Exploración Inicial

Se pone el foco en entender la estructura y los tipos de datos del DataFrame.

### 3.1. Nombres de Columnas y Tipos de Datos
Se obtiene un resumen del DataFrame.

In [ ]:
df_raw.info()

### 3.2. Visualización de Primeras y Últimas Filas
Se muestran las primeras y últimas 5 filas del DataFrame.

In [ ]:
print("--- Primeras 5 filas ---")
display(df_raw.head())

print("\n--- Últimas 5 filas ---")
display(df_raw.tail())

### 3.3. Resumen Estadístico
Se genera un resumen estadístico para todas las columnas

In [ ]:
display(df_raw.describe(include='all').T)

## 4. Análisis de Calidad de Datos

Se investigan los problemas comunes de calidad de datos.

### 4.1. Análisis de Valores Nulos

Se cuantifican y visualizan los valores nulos.

In [ ]:
missing_values = df_raw.isnull().sum()
missing_percentage = (missing_values / len(df_raw)) * 100
missing_df = pd.DataFrame({'count': missing_values, 'percentage': missing_percentage})
missing_df = missing_df[missing_df['count'] > 0].sort_values(by='count', ascending=False)

print(f"Se encontraron {len(missing_df)} columnas con valores nulos.")
display(missing_df)

msno.matrix(df_raw)
plt.show()

### 4.2. Análisis de Duplicados en Clave Primaria (SKU)

Se identifican y cuentan los `sku` que aparecen más de una vez.

In [ ]:
sku_counts = df_raw.sku.value_counts()
duplicated_skus = sku_counts[sku_counts > 1]

print(f"Análisis de duplicados de SKU:")
print(f"- Hay {len(sku_counts)} SKUs únicos de un total de {len(df_raw)} registros.")
print(f"- Se encontraron {len(duplicated_skus)} SKUs con registros duplicados.")

if not duplicated_skus.empty:
    print("\nTop 10 SKUs con más duplicados:")
    display(duplicated_skus.head(10).to_frame(name='count'))

### 4.3. Análisis de Formatos de Fecha
Se examinan los valores únicos de `fecha_actualización` para identificar formatos inconsistentes.

In [ ]:
print(f"La columna 'fecha_actualización' tiene {df_raw.fecha_actualización.nunique()} valores únicos.")
print("\nEjemplos de formatos encontrados:")
display(df_raw.fecha_actualización.value_counts().head(10).to_frame())

### 4.4. Análisis de Columnas Numéricas con Formato Incorrecto

Se inspeccionan las columnas `precio_venta` y `margen` (tipo `object`) para encontrar caracteres no numéricos.

In [ ]:
cols_to_check = ['precio_venta', 'margen']

for col in cols_to_check:
    # Find rows that contain non-numeric characters (excluding decimal point and minus sign)
    problematic_rows = df_raw[df_raw[col].astype(str).str.contains(r'[^0-9.-]', regex=True, na=False)]
    if not problematic_rows.empty:
        print(f"--- Análisis de la columna '{col}' ---")
        print(f"Se encontraron {len(problematic_rows)} filas con caracteres no numéricos.")
        print("Ejemplos de valores problemáticos:")
        display(problematic_rows[col].head().to_frame())

### 4.X. Análisis de Distribución y Outliers en Columnas Numéricas (revisar sección)

Se generan histogramas y diagramas de caja (boxplots) para cada columna numérica.

In [ ]:
numeric_cols = df_raw.select_dtypes(include=np.number).columns

for col in numeric_cols:
    fig, axes = plt.subplots(1, 2, figsize=(15, 4))
    fig.suptitle(f"Análisis de la columna numérica: '{col}'", fontsize=16)

    # Histogram
    sns.histplot(df_raw[col], kde=True, ax=axes[0])
    axes[0].set_title('Distribución (Histograma)')

    # Boxplot
    sns.boxplot(x=df_raw[col], ax=axes[1])
    axes[1].set_title('Diagrama de Caja (Boxplot)')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

### 4.X. Análisis de Columnas Categóricas (revisar sección)

Se examinan los valores únicos y su frecuencia en cada columna de tipo 'object'.

In [ ]:
categorical_cols = df_raw.select_dtypes(include=['object', 'category']).columns

for col in categorical_cols:
    print(f"--- Análisis de la columna: '{col}' ---")
    print(f"Número de valores únicos: {df_raw[col].nunique()}")
    print("Frecuencia de los 10 valores más comunes:")
    display(df_raw[col].value_counts().nlargest(10))
    print("\n" + "="*50 + "\n")

## 5. Limpieza y Transformación de Datos

Se aplica una serie de transformaciones basadas en los hallazgos anteriores para crear un DataFrame limpio.

In [ ]:
# Create a copy for cleaning to preserve the original dataframe
df_cleaned = df_raw.copy()

# --- 1. Limpiar y convertir columnas numéricas ---
def clean_numeric(series):
    return pd.to_numeric(series.astype(str).str.replace(r'[^0-9.-]', '', regex=True), errors='coerce')

df_cleaned['precio_venta'] = clean_numeric(df_cleaned['precio_venta'])
df_cleaned['margen'] = clean_numeric(df_cleaned['margen'])

# --- 2. Convertir columna de fecha ---
df_cleaned['fecha_actualización'] = pd.to_datetime(df_cleaned['fecha_actualización'], errors='coerce', format='mixed')

# --- 3. Limpiar espacios en columnas de texto ---
for col in df_cleaned.select_dtypes(include=['object']).columns:
    df_cleaned[col] = df_cleaned[col].str.strip()

# --- 4. Manejar duplicados de SKU ---
# Strategy: Keep the record with the most recent 'fecha_actualización'.
initial_rows = len(df_cleaned)
df_cleaned = df_cleaned.sort_values('fecha_actualización', ascending=False).drop_duplicates(subset='sku', keep='first')
rows_removed = initial_rows - len(df_cleaned)
print(f"Se eliminaron {rows_removed} filas por duplicados de SKU, conservando la más reciente.")

# --- 5. Manejar valores nulos ---
# Strategy: Fill 'categoria' with 'desconocida'. Drop rows where price or date is null as they are critical.
df_cleaned['categoria'].fillna('desconocida', inplace=True)
rows_before_na_drop = len(df_cleaned)
df_cleaned.dropna(subset=['precio_compra', 'precio_venta', 'fecha_actualización'], inplace=True)
na_rows_removed = rows_before_na_drop - len(df_cleaned)
print(f"Se eliminaron {na_rows_removed} filas por tener valores nulos en columnas críticas.")

print("\n--- Verificación de tipos de datos después de la limpieza ---")
df_cleaned.info()

## 6. Verificación de Integridad de Datos

Se calcula el margen (`precio_venta` - `precio_compra`) y se compara con la columna `margen` original para validar la consistencia.

In [ ]:
df_cleaned['margen_calculado'] = df_cleaned['precio_venta'] - df_cleaned['precio_compra']
df_cleaned['margen_diferencia'] = (df_cleaned['margen'] - df_cleaned['margen_calculado']).abs()

discrepancies = df_cleaned[df_cleaned['margen_diferencia'] > 0.01] # Use tolerance for float precision

print(f"Se encontraron {len(discrepancies)} filas con una diferencia mayor a 0.01 entre el margen reportado y el calculado.")

if not discrepancies.empty:
    print("\nEjemplos de filas con discrepancias:")
    display(discrepancies[['sku', 'precio_compra', 'precio_venta', 'margen', 'margen_calculado']].head())

    plt.figure(figsize=(12, 6))
    sns.histplot(df_cleaned['margen_diferencia'], bins=50, kde=True)
    plt.title('Distribución de la Diferencia Absoluta en el Margen')
    plt.xlabel('Diferencia Absoluta')
    plt.show()

In [ ]:
print("\n--- Resumen Estadístico del DataFrame Limpio ---")
display(df_cleaned.describe(include='all').T)

In [ ]:
print("\n--- Conteo de Nulos Restantes ---")
print(df_cleaned.isnull().sum())

## 7. Análisis de Datos Limpios
Se realizan visualizaciones sobre el DataFrame limpio para evaluar.

### 7.1. Distribución de Variables Numéricas Limpias

In [ ]:
numeric_cols_cleaned = df_cleaned.select_dtypes(include=np.number).columns
df_final = df_cleaned.drop(columns=['margen_calculado', 'margen_diferencia']) # Drop temporary columns for final analysis

for col in ['precio_compra', 'precio_venta', 'margen']:
    fig, axes = plt.subplots(1, 2, figsize=(15, 4))
    fig.suptitle(f"Análisis de la columna numérica limpia: '{col}'", fontsize=16)

    sns.histplot(df_final[col], kde=True, ax=axes[0])
    axes[0].set_title('Distribución (Histograma)')

    sns.boxplot(x=df_final[col], ax=axes[1])
    axes[1].set_title('Diagrama de Caja (Boxplot)')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

### 7.2. Correlación entre Variables Numéricas

In [ ]:
plt.figure(figsize=(10, 8))
correlation_matrix = df_final[['precio_compra', 'precio_venta', 'margen']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Mapa de Calor de Correlación de Variables Numéricas')
plt.show()

## 8. Almacenamiento del DataFrame Limpio

Se guarda el DataFrame limpio y validado en formato Parquet.


In [ ]:
final_df_to_save = df_cleaned.drop(columns=['margen_calculado', 'margen_diferencia'])
output_path = "/content/drive/My Drive/Colab Notebooks/famiq_test/rentabilidad_productos_limpio.parquet"

# Ensure the pyarrow engine is installed for parquet support
!pip install pyarrow -q

final_df_to_save.to_parquet(output_path, index=False, engine='pyarrow')

print(f"DataFrame limpio con {len(final_df_to_save)} filas guardado exitosamente en: {output_path}")